# Technical Assessment Test for IHiS 

sss

## Data Preparation

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
#Reading data from Excel Sheet name "Demographics"
data = pd.read_excel("01_source_files/Technical Test.xls", sheet_name="Demographics")
data.head()

,NRIC,Gender,Age,Marital Status,New NRIC,Coding - Gender,Age Group,Study Number
0,S1510657E,M,39,Divorced,NaN,NaN,NaN,NaN
1,SN5484655,F,68,Single,NaN,NaN,NaN,NaN
2,S7705341S,M,1,Married,NaN,NaN,NaN,NaN
3,SA9787544,M,35,Married,NaN,NaN,NaN,NaN
4,SA7766801,F,91,Married,NaN,NaN,NaN,NaN


In [3]:
# Quick review of the dataset details, types etc
data.info()

#To understand the size of the dataset
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NRIC             117 non-null    object 
 1   Gender           117 non-null    object 
 2   Age              117 non-null    int64  
 3   Marital Status   117 non-null    object 
 4   New NRIC         0 non-null      float64
 5   Coding - Gender  0 non-null      float64
 6   Age Group        0 non-null      float64
 7   Study Number     0 non-null      float64
dtypes: float64(4), int64(1), object(3)
memory usage: 7.4+ KB


(117, 8)

In [4]:
data.duplicated().sum()

5

In [5]:
data.isna().sum()

NRIC                 0
Gender               0
Age                  0
Marital Status       0
New NRIC           117
Coding - Gender    117
Age Group          117
Study Number       117
dtype: int64

## New NRIC

In [6]:
#Noticed some empty space in between, removed them by using .replace()
data["NRIC"] = data.NRIC.str.replace(" ","")
data.head()

,NRIC,Gender,Age,Marital Status,New NRIC,Coding - Gender,Age Group,Study Number
0,S1510657E,M,39,Divorced,NaN,NaN,NaN,NaN
1,SN5484655,F,68,Single,NaN,NaN,NaN,NaN
2,S7705341S,M,1,Married,NaN,NaN,NaN,NaN
3,SA9787544,M,35,Married,NaN,NaN,NaN,NaN
4,SA7766801,F,91,Married,NaN,NaN,NaN,NaN


In [159]:
nric_list = pd.DataFrame(data["NRIC"])
print(nric_list.shape)
nric_list.head()

(117, 1)


,NRIC
0,S1510657E
1,SN5484655
2,S7705341S
3,SA9787544
4,SA7766801


In [160]:
new_preprocessed_nric = []

for ic in nric_list["NRIC"]:
    if (ic[0] and ic[-1]).isalpha() == True:
        new_preprocessed_nric.append(ic)
    else:

        start_char = []
        end_char = []
        numbers = []

        if (ic[0]).isdigit() == False:
            start_char.append(ic[0])
        if (ic[1]).isdigit() == False:
            end_char.append(ic[1])
        if (ic[2:]).isdigit() == True:
            numbers.append(ic[2:])

        new_preprocessed_nric.append("".join((start_char+numbers+end_char)))        
            
            
            



In [162]:
new_preprocessed_nric.head()

AttributeError: 'list' object has no attribute 'head'

## Coding - Gender

In [ ]:
#Function to label/code Gender Group

def gender_code(gender_cols):
   if gender_cols == "F" :
      return 1
   else:
      return 2


In [ ]:
#Apply the function with the return values to Gender Group Column
data["Coding - Gender"] = data["Gender"].apply (lambda gender_cols: gender_code(gender_cols))

In [ ]:
data

In [ ]:
np.sort(data["Coding - Gender"].unique())

## Age Group

In [ ]:
#Function to define and categories Age Group

def age_group(age_cols):
   if age_cols <= 9 :
      return "G1"
   if age_cols >= 10 and age_cols <= 19:
      return "G2"
   if age_cols >= 20 and age_cols <= 29:
      return "G3"
   if age_cols >= 30 and age_cols <= 39:
      return "G4"  
   if age_cols >= 40 and age_cols <= 49:
      return "G5"
   if age_cols >= 50 and age_cols <= 59:
      return "G6"
   if age_cols >= 60 and age_cols <= 69:
      return "G7"  
   if age_cols >= 70 and age_cols <= 79:
      return "G8"
   if age_cols >= 80 and age_cols <= 89:
      return "G9"
   if age_cols >= 90:
      return "G0"        


In [ ]:
#Apply the function with the return values to Age Group Column
data["Age Group"] = data["Age"].apply (lambda age_cols: age_group(age_cols))
data.head(15)

In [ ]:
np.sort(data["Age Group"].unique())

## Study Number

In [ ]:
data["Study Number"] = data["Age Group"] + " - " + data["Coding - Gender"].astype(str)
data.head(15)

## Worksheet: Study Data

### Study Data Worksheet

In [ ]:
#Reading data from Excel Sheet name "Study Data"
study_data =  pd.read_excel("01_source_files/Technical Test.xls", sheet_name="Study Data")
study_data.head()

In [ ]:
study_data.info()
print(study_data.shape)

### Extra Information Worksheet

In [ ]:
#Reading data from Excel Sheet name "Extra information"
data_extra =  pd.read_excel("01_source_files/Technical Test.xls", sheet_name="Extra information")
data_extra.head()

In [ ]:
data_extra.info()
print(data_extra.shape)

### Combination of Worksheets for Study Data

In [ ]:
#Merging/combine the data in Demographics worksheet and Extra Information worksheet
demo_n_extra = data.merge(data_extra, how ="left", on="NRIC")

In [ ]:
demo_n_extra.head(10)

In [ ]:
demo_n_extra.shape

In [ ]:
data_extra[data_extra["NRIC"] == "SA9787544"]

In [ ]:
demo_n_extra["Ethnic Group"].isna().sum()

### Final Combine to Study Data (Not Yet Complete!!)

In [ ]:
#Merge or Join data based on columns name

In [ ]:
new_data = study_data.merge(data_extra, how ="left", left_on="Old NRIC", right_on="NRIC")

In [ ]:
new_data.head(10)

In [ ]:
new_data.shape

## Worksheet: Exception List

### List of unique NRIC

In [ ]:
# # Number of Unique NRIC (New Formatted)
# unique_nric = pd.DataFrame(data["NRIC"].value_counts().loc[lambda x : x>1].rename_axis("NRIC").reset_index(name="Counts"))
# unique_nric.index = unique_nric.index + 1

# unique_nric.index.name = "id"
# unique_nric

## *Change to "New NRIC" later*

In [ ]:
# Number of Unique NRIC (New Formatted)
unique_nric = pd.DataFrame(data["NRIC"].value_counts().rename_axis("NRIC").reset_index(name="Counts"))
unique_nric.index = unique_nric.index + 1
unique_nric = unique_nric.rename_axis("S/N").reset_index()

unique_nric

In [ ]:
unique_nric_final = pd.DataFrame(unique_nric[["S/N","NRIC"]]).set_index("S/N")
unique_nric_final

In [ ]:
unique_nric_final.to_excel("02_exported_worksheet/unique_num_nric.xlsx")

In [ ]:
#Number of NRIC under Demographics that are found in Extra Information workshee.

nric_notin = data[~data["NRIC"].isin(data_extra["NRIC"])]
print(nric_notin.shape)
nric_notin

## Worksheet: Pivot Table

In [ ]:
# demo_n_extra["Age Group"])["Age Group"].value_counts()

In [ ]:
# pd.pivot_table(data=demo_n_extra,index=["Age Group", "Gender"], values=["Marital Status"],aggfunc=np.sum)



In [ ]:
# p_tabel = pd.pivot_table(data=demo_n_extra, index=["Age Group", "Gender"], values=["Marital Status"], aggfunc=np.sum)
# p_tabel

In [ ]:
demo_n_extra["Age Group"].value_counts()

In [ ]:
demo_n_extra["Gender"].value_counts()